**Module 10 - Task 1**

My coding approach for this task is based on functions to ensure reproducibility. While I use Serie A 2023/2024 as an example, this code is versatile and can be applied to any available competition and season by simply changing the *competition_id* and *season* variable values.

# API Requests

In [10]:
# Importing the necessary libraries

import pandas as pd
import requests

With my first API request i want to find out the available competitions

In [11]:
# Define the url to get the data of available competitions

url = "http://api.football-data.org/v4/competitions/"

# Request the data from the api

response = requests.get(url)

# store the response

data = response.json()

# Create a dataframe to store the response

competitions = pd.json_normalize(data['competitions'])

# Take a first look at the data

competitions.head(1)

,id,name,code,type,emblem,plan,numberOfAvailableSeasons,lastUpdated,area.id,area.name,...,currentSeason.winner.shortName,currentSeason.winner.tla,currentSeason.winner.crest,currentSeason.winner.address,currentSeason.winner.website,currentSeason.winner.founded,currentSeason.winner.clubColors,currentSeason.winner.venue,currentSeason.winner.lastUpdated,currentSeason
0,2176,Africa Cup,AC,CUP,https://crests.football-data.org/ac.png,TIER_FOUR,1,2024-01-13T11:34:43Z,2001,Africa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# Check the unique available competitions

competitions.name.unique()

array(['Africa Cup', 'AFC Champions League', 'WC Qualification CAF',
       'Primera B Nacional', 'Liga Profesional', 'Copa Liga Profesional',
       'Supercopa Argentina', 'WC Qualification AFC', 'A League',
       'FFA Cup', 'Erste Liga', 'Bundesliga', 'Playoffs 1/2', 'ÖFB Cup',
       'Coupe de Belgique', 'Division 1B', 'Jupiler Pro League',
       'Playoffs', 'Supercoupe de Belgique', 'LFPB', 'Premier Liga',
       'Copa do Brasil', 'Série D', 'Série C',
       'Campeonato Brasileiro Série B', 'Campeonato Brasileiro Série A',
       'Kupa na Bulgarija', 'A PFG', 'Canadian Championship',
       'Supercopa de Chile', 'Primera División', 'Chinese Super League',
       'Liga Postobón', 'Superliga de Colombia', 'Prva Liga',
       'Synot Liga', 'DBU Pokalen', 'Superliga', 'Euro League - Playoff',
       'Copa Pilsener Serie A', 'Football League Cup',
       'FA Community Shield', 'FA Cup', 'National League', 'League Two',
       'League One', 'Championship', 'Premier League',
       'FA

In [13]:
# I decide to use Serie A (Italy) as my case study for the task, i call the row of Serie A to check its id

competitions[competitions['name']=='Serie A']

,id,name,code,type,emblem,plan,numberOfAvailableSeasons,lastUpdated,area.id,area.name,...,currentSeason.winner.shortName,currentSeason.winner.tla,currentSeason.winner.crest,currentSeason.winner.address,currentSeason.winner.website,currentSeason.winner.founded,currentSeason.winner.clubColors,currentSeason.winner.venue,currentSeason.winner.lastUpdated,currentSeason
95,2019,Serie A,SA,LEAGUE,https://crests.football-data.org/SA.png,TIER_ONE,92,2022-03-20T09:16:43Z,2114,Italy,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# Select competition id

competition_id = 2019 # change to any competition id

# Select season

season = 2023 # change to any season

In [15]:
## Define the url for our selected competition and season, set my credentials, send the request and store the data in the competition dataframe

url = f"http://api.football-data.org/v4/competitions/{competition_id}/matches?season={season}"

headers = {'X-Auth-Token' : '608cc9447fde4b9b8f0c1f78f4fc3ed9'}

response = requests.request('GET',url,headers = headers)

data = response.json()

competition = pd.json_normalize(data['matches'])

# Metric A calculation

**Metric A:** The first metric is the calculation of the times the home team was leading in HT and managed to keep the win and the percentage of total.

I will create a function to perform these calculations

In [16]:
## Define function to calculate the home wins number and conversion rate when winning at halftime

def calculate_ht_home_wins(competition):

  # Create a new column 'ht_ft_win' indicating if home team was winning at halftime and won the game

  competition['ht_ft_home_win'] = ((competition['score.halfTime.home'] > competition['score.halfTime.away']) & (competition['score.winner'] == 'HOME_TEAM')).astype(int)

  # Create a new column 'ht_win' indicating if home team was winning at halftime regardless the end result

  competition['ht_home_win'] = (competition['score.halfTime.home'] > competition['score.halfTime.away']).astype(int)

  # Create a new data frame, grouped by home team name and with ht_win and ht_ft_win values aggregated using sum function.

  ht_home_wins = competition.groupby('homeTeam.name')[['ht_home_win', 'ht_ft_home_win']].sum().reset_index()

  # Calculate the percentage each team managed to win the game after winning in half time

  ht_home_wins['ht_ft_home_win_pct'] = round(ht_home_wins['ht_ft_home_win'] / (ht_home_wins['ht_home_win']) * 100, 2)

  # We are sorting the values by the success rate in descending order

  ht_home_wins = ht_home_wins.sort_values(by='ht_ft_home_win_pct',ascending = False).reset_index(drop=True)

  # Rename the team column

  ht_home_wins = ht_home_wins.rename(columns={'homeTeam.name': 'team'})

  return ht_home_wins

In [17]:
## Apply the function

# Calculate home wins when winning at halftime

ht_home_wins = calculate_ht_home_wins(competition)

# Display the results

ht_home_wins

,team,ht_home_win,ht_ft_home_win,ht_ft_home_win_pct
0,Torino FC,4,4,100.00
1,SS Lazio,4,4,100.00
2,Empoli FC,4,4,100.00
3,Juventus FC,4,4,100.00
4,AC Milan,11,10,90.91
5,Bologna FC 1909,8,7,87.50
6,Atalanta BC,11,9,81.82
7,Frosinone Calcio,4,3,75.00
8,SSC Napoli,4,3,75.00
9,Genoa CFC,8,6,75.00


# Metric B calculation



**Metric B:** The next part is to calculate the percentage of turnarounds for each team when they are trailing at half time.

I will create four functions to add some necessary columns, split the data to home & away teams and finally calculate the turnarounds number and pct.

In [18]:
## Define the functions to add the needed information to the dataframe to perform our calculations

# Function to add the columns indicating if home or away team was trailing in half time

def add_ht_trailing_columns(df):

    df['home_ht_trailing'] = (df['score.halfTime.away'] > df['score.halfTime.home']).astype(int)
    df['away_ht_trailing'] = (df['score.halfTime.away'] < df['score.halfTime.home']).astype(int)

    return df

# Function to add the columns indicating if home or away team did a turnaround

def add_turnaround_columns(df):

    df['home_turnaround'] = ((df['score.halfTime.away'] > df['score.halfTime.home']) & (df['score.winner'] == 'HOME_TEAM')).astype(int)
    df['away_turnaround'] = ((df['score.halfTime.away'] < df['score.halfTime.home']) & (df['score.winner'] == 'AWAY_TEAM')).astype(int)

    return df

# Function to split the data to home and away team stats

def create_home_away_df(df, team_type):

    if team_type == 'home':
        df_team = df[['homeTeam.name', 'id', 'home_ht_trailing', 'home_turnaround', 'score.winner']].copy() #create a new dataframe with selected columns only
        df_team['home_wins'] = (df_team['score.winner'] == 'HOME_TEAM').astype(int) # add the number of home wins
        df_team['home_draws'] = (df_team['score.winner'] == 'DRAW').astype(int) # add the number of home draws -- for the bonus task
        df_team = df_team.groupby('homeTeam.name').agg( # group by team and perform aggregations
                       home_wins = ('home_wins', 'sum'),
                       home_draws = ('home_draws', 'sum'),
                       home_ht_trailing = ('home_ht_trailing', 'sum'),
                       home_turnaround = ('home_turnaround', 'sum'),
                       home_games = ('id', 'count')).reset_index()
        df_team = df_team.rename(columns={'homeTeam.name': 'team'}) # rename the homeTeam.name column
    elif team_type == 'away':
        df_team = df[['id','awayTeam.name', 'away_ht_trailing', 'away_turnaround', 'score.winner']].copy()
        df_team['away_wins'] = (df_team['score.winner'] == 'AWAY_TEAM').astype(int) # add the number of away wins
        df_team['away_draws'] = (df_team['score.winner'] == 'DRAW').astype(int) # add the number of away draws -- for the bonus task
        df_team = df_team.groupby('awayTeam.name').agg( # group by team and perform aggregations
                       away_wins = ('away_wins', 'sum'),
                       away_draws = ('away_draws', 'sum'),
                       away_ht_trailing = ('away_ht_trailing', 'sum'),
                       away_turnaround = ('away_turnaround', 'sum'),
                       away_games = ('id', 'count')).reset_index()
        df_team = df_team.rename(columns={'awayTeam.name': 'team'}) # rename the homeTeam.name column

    return df_team

# Function to merge the home and away dataframes and calculate the turnaround metrics

def calculate_turnarounds(home, away):

    # merge home & away df

    home_and_away = pd.merge(home, away, on='team', how='inner').reset_index(drop=True)

    # create a new df with selected columns

    turnarounds = home_and_away[['team', 'home_ht_trailing', 'home_turnaround', 'away_ht_trailing', 'away_turnaround']].copy()

    # calculate the total times each team was trailing in HT

    turnarounds['ht_trailing'] = turnarounds['home_ht_trailing'] + turnarounds['away_ht_trailing']

    # calculate the total number of turnarounds for each team

    turnarounds['total_turnarounds'] = turnarounds['home_turnaround'] + turnarounds['away_turnaround']

    # calculate the turnaround percentage for each team

    turnarounds['turnarounds_pct'] = round((turnarounds['total_turnarounds'] / turnarounds['ht_trailing']) * 100, 2)

    # drop unecessary columns and sort values by the pct

    turnarounds = turnarounds[['team', 'ht_trailing', 'total_turnarounds', 'turnarounds_pct']].sort_values(by='turnarounds_pct', ascending=False).reset_index(drop=True)

    return turnarounds

In [19]:
## Apply the functions to produce our results

# Add columns using add_ht_trailing_columns & add_turnaround_columns functions to calculate the turnarounds

competition = add_ht_trailing_columns(competition)
competition = add_turnaround_columns(competition)

# Split the data frame to home teams and away teams data using home & away functions

home_data = create_home_away_df(competition, 'home')
away_data = create_home_away_df(competition, 'away')

# Calculate the turnarounds

turnaround_data = calculate_turnarounds(home_data, away_data)

# Show the results

turnaround_data

,team,ht_trailing,total_turnarounds,turnarounds_pct
0,FC Internazionale Milano,4,2,50.00
1,Genoa CFC,8,3,37.50
2,Bologna FC 1909,10,3,30.00
3,AS Roma,10,2,20.00
4,Atalanta BC,11,2,18.18
5,Frosinone Calcio,16,2,12.50
6,Cagliari Calcio,18,2,11.11
7,Juventus FC,10,1,10.00
8,SSC Napoli,11,1,9.09
9,US Sassuolo Calcio,12,1,8.33


# Bonus Question

**Bonus Question:** Calculate the league table for the selected competition.

I will create a function that utilizes the previously created *home_data* and *away_data* DataFrames. This function will merge the two DataFrames and perform all necessary calculations to aggregate wins, draws, losses, and games played for each team, ultimately producing the league table.

In [20]:
## Calculate the total points for each team

def calculate_league_table(home_data, away_data):

  # Merge home_data & away_data DataFrames

  full_data = pd.merge(home_data[['team','home_wins','home_draws','home_games']], away_data[['team','away_wins','away_draws','away_games']],on='team', how='inner')

  # Calculate total games played, wins, draws & losses for each team

  full_data['games_played'] = full_data['home_games'] + full_data['away_games']
  full_data['win'] = full_data['home_wins'] + full_data['away_wins']
  full_data['draw'] = full_data['home_draws'] + full_data['away_draws']
  full_data['loss'] = full_data['games_played'] - (full_data['win'] + full_data['draw'])

  # Calculate points

  full_data['points'] = (full_data['win'] * 3) + (full_data['draw'] * 1)

  # Sort values on points

  full_data = full_data.sort_values(by='points', ascending = False).reset_index(drop=True)

  # Add the league position info

  full_data['position'] = full_data.index+1

  # Keep selected columns

  league_table = full_data[['position','team','games_played', 'points','win', 'draw','loss']]

  return league_table

In [21]:
## Apply the function to create the league table

# Calculate league table

league_table = calculate_league_table(home_data, away_data)

# Present league table

league_table

,position,team,games_played,points,win,draw,loss
0,1,FC Internazionale Milano,38,94,29,7,2
1,2,AC Milan,38,75,22,9,7
2,3,Juventus FC,38,71,19,14,5
3,4,Atalanta BC,38,69,21,6,11
4,5,Bologna FC 1909,38,68,18,14,6
5,6,AS Roma,38,63,18,9,11
6,7,SS Lazio,38,61,18,7,13
7,8,ACF Fiorentina,38,60,17,9,12
8,9,Torino FC,38,53,13,14,11
9,10,SSC Napoli,38,53,13,14,11


In [22]:
## Insights part

# Merge all the info in one table

insights_table = pd.merge(league_table, ht_home_wins, on='team', how='inner').merge(turnaround_data, on='team', how='inner')

# Show the results

insights_table

,position,team,games_played,points,win,draw,loss,ht_home_win,ht_ft_home_win,ht_ft_home_win_pct,ht_trailing,total_turnarounds,turnarounds_pct
0,1,FC Internazionale Milano,38,94,29,7,2,16,12,75.00,4,2,50.00
1,2,AC Milan,38,75,22,9,7,11,10,90.91,5,0,0.00
2,3,Juventus FC,38,71,19,14,5,4,4,100.00,10,1,10.00
3,4,Atalanta BC,38,69,21,6,11,11,9,81.82,11,2,18.18
4,5,Bologna FC 1909,38,68,18,14,6,8,7,87.50,10,3,30.00
5,6,AS Roma,38,63,18,9,11,8,6,75.00,10,2,20.00
6,7,SS Lazio,38,61,18,7,13,4,4,100.00,5,0,0.00
7,8,ACF Fiorentina,38,60,17,9,12,8,5,62.50,13,1,7.69
8,9,Torino FC,38,53,13,14,11,4,4,100.00,3,0,0.00
9,10,SSC Napoli,38,53,13,14,11,4,3,75.00,11,1,9.09


**Serie A 2023/2024 insights**

*   Teams in higher positions generally exhibit a higher conversion percentage of wins when leading at halftime. However, we also observe high conversion rates among teams in lower positions. Thus, the frequency of this occurrence is more significant than the conversion rate percentage.

*   Turnarounds are not strongly correlated with league success because stronger teams typically do not find themselves trailing at halftime as often.

